In [1]:
!%pip install --upgrade langchain
!%pip install --upgrade langchain-openai
!%pip install -qU langchain-core langchain-openai
!%pip install --upgrade openai
!%pip install azure-search
%pip install -r ../../../requirements.txt

/bin/bash: line 1: fg: no job control
/bin/bash: line 1: fg: no job control
/bin/bash: line 1: fg: no job control
/bin/bash: line 1: fg: no job control
/bin/bash: line 1: fg: no job control
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import json
from openai import AzureOpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.models import VectorizedQuery, VectorQuery
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient

In [3]:

from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("This lab exercise will use the following values:")
    print("Azure OpenAI Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
    print("Azure AI Search: " + os.getenv("AZURE_AI_SEARCH_SERVICE_NAME"))
else: 
    print("No file .env found")

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_version = os.getenv("OPENAI_API_VERSION")
azure_openai_completion_deployment_name = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
azure_openai_embedding_deployment_name = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")
azure_ai_search_name = os.getenv("AZURE_AI_SEARCH_SERVICE_NAME")
azure_ai_search_endpoint = os.getenv("AZURE_AI_SEARCH_ENDPOINT")
azure_ai_search_index_name = os.getenv("AZURE_AI_SEARCH_INDEX_NAME")
azure_ai_search_api_key = os.getenv("AZURE_AI_SEARCH_API_KEY")

This lab exercise will use the following values:
Azure OpenAI Endpoint: https://oai-mtpchatbot-dev-frcent-001.openai.azure.com/
Azure AI Search: srch-mtpchatbot-dev-frcent-001


In [15]:
import json
from openai import AzureOpenAI
import os

endpoint ="https://oai-mtpchatbot-dev-frcent-001.openai.azure.com/"
deployment='oaidepl-mtpchatbot-dev-frcent-003'

search_client = SearchClient(
    endpoint=azure_ai_search_endpoint, 
    index_name=azure_ai_search_index_name, 
    credential=azure_ai_search_api_key
)

client = AzureOpenAI(
    azure_endpoint=azure_openai_endpoint,
    api_key="a74bc358eb354a7590d19d43fedc999a",
    api_version="2023-12-01-preview",
    )



In [5]:
from langchain_openai import AzureOpenAIEmbeddings

azure_openai_embeddings = AzureOpenAIEmbeddings(
    azure_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"),
    openai_api_version = os.getenv("OPENAI_EMBEDDING_API_VERSION"),
    model= os.getenv("AZURE_OPENAI_EMBEDDING_MODEL"),
)

from langchain_community.vectorstores.azuresearch import AzureSearch

index_name: str = "mtp-chatbot-index"
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=os.getenv("AZURE_AI_SEARCH_ENDPOINT"),
    azure_search_key=os.getenv("AZURE_AI_SEARCH_API_KEY"),
    index_name=index_name,
    embedding_function=azure_openai_embeddings.embed_query,
)

# Perform a similarity search testwise
results = vector_store.similarity_search(
    query="Wie schließe ich einen TruckLive Vertrag ab?",
    k=3, # forgot what this was -> k nearest neighbour?
    search_type="hybrid",
    # adjust relevance with score_threshold= xy,
)



print(results[0].page_content)

vector_search_configuration is not a known attribute of class <class 'azure.search.documents.indexes.models._index.SearchField'> and will be ignored


TRUCKLIVE: KURZANLEITUNG VERTRAGSABSCHLUSS



3

Schritt 3:
Stimmen Sie dem TruckLive Vertrag zu. (     )
Klicken Sie anschließend auf „Bestätigen
und Weiter“. (     )

4

5

TRUCKLIVE: KURZANLEITUNG VERTRAGSABSCHLUSS



4

Schritt 4:
Definieren Sie einen Zeitpunkt, ab dem TruckLive
für zukünftige Fahrzeuge aktiviert werden soll. (     )
Wählen Sie danach eine Kontaktperson aus dem
eigenen Unternehmen aus, welche die Meldungen 
zu TruckLive bekommen soll. (     )


In [6]:
def get_embeddings(text: str):
    # There are a few ways to get embeddings. This is just one example.
    import openai

    open_ai_endpoint = os.getenv(azure_openai_endpoint)
    open_ai_key = os.getenv(azure_openai_api_key)

    client = openai.AzureOpenAI(
        azure_endpoint=open_ai_endpoint,
        api_key=open_ai_key,
        api_version="2023-12-01-preview",

    )
    embedding = client.embeddings.create(input=[text], model=azure_openai_embedding_deployment_name)
    return embedding.data[0].embedding


def hybrid_search(query):
    # [START simple_hybrid_search]

    search_client = SearchClient(azure_ai_search_endpoint, index_name, AzureKeyCredential(azure_ai_search_api_key))
    vector_query = VectorizedQuery(vector=get_embeddings(query), k_nearest_neighbors=3, fields="vector")

    results = search_client.search(
        search_text=query,
        query_type= "semantic",
        vector_queries=[vector_query],
        semantic_configuration_name= "mtp-chatbot-semantic-configuration",
        select=["title", "chunk", "url", "downloadable"],
        top=5,
    )

    n = 1
    results_for_prompt = ""
    for result in results:
        if result['downloadable'] == "false":
            results_for_prompt += f"Result {result['title']}: {result['chunk']}\n"
        else:
            results_for_prompt += f"Result {result['title']}: {result['chunk']}:" + " URL: " + f"{result['url']}: {result['downloadable']}\n"
        n += 1
    
    return results_for_prompt

print(hybrid_search("What is My TruckPoint ?"))




Result FAQ_MTP en-GB_041223.txt: vehicle is still in use (in your active fleet) or no longer in your possession (move to archive). The goal is to have no vehicles in the unclassified fleet.
	Fleet	Can I integrate vehicles from other brands?	Yes, My TruckPoint is a platform that can be used to manage your entire vehicle fleet, i.e. third-party products can also be integrated. You can add vehicles to the list of active fleets using the VIN. The full range of functions is not available for vehicles added in this way.
Result versch. Wissensdokumente MTP aus Internet und Retail Panel.docx: die dazugehörigen Dienste, Verträge und Informationen einfach und übersichtlich verwalten können. Es unterstützt Sie in der Kommunikation mit Ihrem Verkäufer und bei der Erledigung der täglichen Aufgaben.
Mit My TruckPoint sind Sie immer über den aktuellen Stand Ihrer Neufahrzeugbestellungen informiert und erhalten jederzeit Einblicke in den geplanten Lieferstatus.
Mit dem Kundenportal haben Sie alle wich

In [38]:

company_vehicles = [
        {
            "vin": 123456789,
            "trucklive_ready": True,
            "trucklive_activation_status": False,
            "active_servicecontract": True
        },
        {
            "vin": 9283740,
            "trucklive_ready": True,
            "trucklive_activation_status": True,
            "active_servicecontract": True
        },
        {
            "vin": 1234,
            "trucklive_ready": False,
            "trucklive_activation_status": False,
            "active_servicecontract": False
        },
        {
            "vin": 123456,
            "trucklive_ready": True,
            "trucklive_activation_status": True,
            "active_servicecontract": False
        },
]




# Function to generate embeddings for title and content fields, also used for query embeddings
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def generate_embeddings(text):
    response = client.embeddings.create(input=text, model=azure_openai_embedding_deployment_name)
    embeddings = response.data[0].embedding
    return embeddings

def get_trucklive_status(vin: int):
    if any(d['vin'] == vin for d in company_vehicles):
        vehicle = list(filter(lambda vehicle: vehicle['vin'] == vin, company_vehicles))
        if vehicle[0]['trucklive_ready']:
            return str(vehicle[0]['trucklive_activation_status'])
        else: return "TruckLive is not available for this vehicle, possibly because it's an older model"      
    else: 
        return "vehicle could not be found or does not belong to your company. Please try again."

def get_current_contracts(user, password):
    if user == "yannik" and password == 123:
        return "TruckLive, Fleetboard, Uptime"
    else: return "None"

def search_sources(query):
    # [START simple_hybrid_search]
    endpoint ="https://oai-mtpchatbot-dev-frcent-001.openai.azure.com/"
    deployment2='oaidepl-mtpchatbot-dev-frcent-003'
    print(query)
    messages2 = []
    messages2.append({"role": "system", "content": """"
    If the requested information is not available in the retrieved data, please answer with something like that: 
    I'm sorry, but this question about XY is out of the scope of my knowledge as I am an AI assistent designed to provide information related to My TruckPoint. 
    If you have questions to the usage of My TruckPoint, TruckLive or other related topics, I would be happy to assist you.
    If the user asks for a term that you find SIMILIAR but NOT EQUAL in your source data, ask the user first whether he really means this term. If the user confirms, explain the term. If the user doesn't confirm, tell that you can unfortunately not help on this and ask for a new question.
    
    You are a friendly and professional support chatbot for customers of the application MyTruckpoint. 
    Your task is to answer questions about the use of MyTruckpoint and all associated components.
    Only tell that it is not available in his country.
    Your answer shouldn't be longer then round about 100 words. If the user asks a question in language XY, answer to him in language XY.
    """})
    messages2.append({"role": "user", "content": query})

    client2 = AzureOpenAI(
        base_url=f"{endpoint}/openai/deployments/{deployment2}/extensions",
        api_key="a74bc358eb354a7590d19d43fedc999a",
        api_version="2023-12-01-preview",
        )
    
    completion = client2.chat.completions.create(
        model=deployment2,
        messages=messages2,
        temperature=0.2,
        extra_body={
            "dataSources": [
                {
                    "type": "AzureCognitiveSearch",
                    "parameters": {
                        "endpoint": os.environ["AZURE_AI_SEARCH_ENDPOINT"],
                        "key": os.environ["AZURE_AI_SEARCH_API_KEY"],
                        "indexName": "mtp-chatbot-index",
                        "queryType": "vectorSemanticHybrid",
                        "semanticConfiguration": "mtp-chatbot-semantic-configuration",
                        "embeddingDeploymentName":"oaidepl-mtpchatbot-dev-frcent-002",
                        "fieldsMapping": {
                            "filepathField": "downloadable"
                        }
                    }
                }
            ]
        }
    )
    return completion.choices[0].message.content

def check_active_service_contract(vin: int):
    if any(d['vin'] == vin for d in company_vehicles):
        vehicle = list(filter(lambda vehicle: vehicle['vin'] == vin, company_vehicles))
        if vehicle[0]['active_servicecontract']:
            return "It seems there is an active service contract for this vehicle"
        else:
            return "There is no active Service Contract for this vehicle. Do you want to know how to conclude one?"
    else: 
        return "vehicle could not be found or does not belong to your company. Please try again."
    
def check_number_plate(vin: int):
    if any(d['vin'] == vin for d in company_vehicles):
        vehicle = list(filter(lambda vehicle: vehicle['vin'] == vin, company_vehicles))
        if vehicle[0]['active_servicecontract']:
            return "It seems there is an active service contract for this vehicle so you should be able to see the number plate. Please contact the support for help."
        else:
            return "To show the number plate automatically you need an active service contract. Please conclude one or register the number plate manually."
    else: 
        return "vehicle could not be found or does not belong to your company. Please try again."
    


def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = []
    messages.append({"role": "system", "content": """
    You are a friendly and professional support chatbot for customers of the application My TruckPoint. 
    Your task is to answer questions about the use of MyTruckpoint and all associated components.
    If the requested information is not available in the retrieved data, please answer with something like that: 
    I'm sorry, but this question about XY is out of the scope of my knowledge as I am an AI assistent designed to provide information related to My TruckPoint. 
    If you have questions to the usage of My TruckPoint, TruckLive or other related topics, I would be happy to assist you.
    If the user asks for a term that you find SIMILIAR but NOT EQUAL in your source data, ask the user first whether he really means this term. If the user confirms, explain the term. If the user doesn't confirm, tell that you can unfortunately not help on this and ask for a new question.
    Your answer shouldn't be longer then round about 100 words. If the user asks a question in language XY, answer to him in language XY.
    Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.
    
     """})
    messages.append({"role": "user", "content": "what is the status of my truck with the vin 1234 and 123456"})
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_trucklive_status",
                "description": "Returns the TruckLive Activation Status of a vehicle given the VIN (Vehicle Identification Number).",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "vin": {
                            "type": "integer",
                            "description": "the vehicle identification number",
                        },
                    },
                    "required": ["vin"],
                },
            }
        },
                {
            "type": "function",
            "function": {
                "name": "get_current_contracts",
                "description": "Return the current contracts the user is subscribed to given the user name and password",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "user": {
                            "type": "string",
                            "description": "the user name",
                        },
                        "password": {
                            "type": "string",
                            "description": "the password to the corresponding user name",
                        },
                    },
                    "required": ["user", "password"],
                },
            }
        },
        {
            "type": "function",
            "function": {
                "name": "search_sources",
                "description": """
                Retrieve information about My TruckPoint, TruckLive and everything related from an Azure AI search index.
                """,
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The query string to search for relevant data about My TruckPoint and everything",
                        },
                    },
                    "required": ["query"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "check_active_service_contract",
                "description": "Returns if there is an acitve Service Contract for a vehicle given the VIN (Vehicle Identification Number).",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "vin": {
                            "type": "integer",
                            "description": "the vehicle identification number",
                        },
                    },
                    "required": ["vin"],
                },
            }
        },
        {
            "type": "function",
            "function": {
                "name": "check_number_plate",
                "description": "To show the number plate there must be an acitve service contract for a Truck with a given VIN, therefore use this function if a User asks why there is no number plate showing for a certain Truck in My TruckPoint.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "vin": {
                            "type": "integer",
                            "description": "the vehicle identification number",
                        },
                    },
                    "required": ["vin"],
                },
            }
        },
    ]
    response = client.chat.completions.create(
        model=deployment,
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_trucklive_status": get_trucklive_status,
            "get_current_contracts": get_current_contracts,
            "search_sources": search_sources,
            "check_active_service_contract": check_active_service_contract,
            "check_number_plate": check_number_plate,
        }

        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            if function_to_call == search_sources:
                print("erster Fall")
                function_args = json.loads(tool_call.function.arguments)
                function_response = function_to_call(messages[1]["content"])
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    }
                )
            else:
                print("zweiter Fall")
                function_args = json.loads(tool_call.function.arguments)
                function_response = function_to_call(**function_args)
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    }
                )
                # extend conversation with function response
        print(messages)
        second_response = client.chat.completions.create(
            model=deployment,
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response.model_dump_json(indent=2)
    else:
        return response.model_dump_json(indent=2)

print(run_conversation())


zweiter Fall
zweiter Fall
[{'role': 'system', 'content': "\n    You are a friendly and professional support chatbot for customers of the application My TruckPoint. \n    Your task is to answer questions about the use of MyTruckpoint and all associated components.\n    If the requested information is not available in the retrieved data, please answer with something like that: \n    I'm sorry, but this question about XY is out of the scope of my knowledge as I am an AI assistent designed to provide information related to My TruckPoint. \n    If you have questions to the usage of My TruckPoint, TruckLive or other related topics, I would be happy to assist you.\n    If the user asks for a term that you find SIMILIAR but NOT EQUAL in your source data, ask the user first whether he really means this term. If the user confirms, explain the term. If the user doesn't confirm, tell that you can unfortunately not help on this and ask for a new question.\n    Your answer shouldn't be longer then r